In [ ]:
!pip install -q transformers accelerate sentencepiece tqdm


In [ ]:
import torch

print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


GPU available: True
GPU name: Tesla T4


In [ ]:
import sqlite3

# ==== CONFIG: CHANGE THESE IF NEEDED ====
DB_PATH = "/content/StockNewsProject/finviz_news.db"
TABLE_NAME = "finviz_news_highfreq_table"   # your subset table
ID_COL = "id"
TITLE_COL = "title"
EXPANDED_COL = "expanded"
# ========================================

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Ensure table exists (will raise if it doesn't, which is good feedback)
cur.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_NAME}';")
row = cur.fetchone()
if not row:
    raise RuntimeError(f"Table {TABLE_NAME} does not exist in {DB_PATH}.")

# Ensure 'expanded' column exists
cur.execute(f"PRAGMA table_info({TABLE_NAME});")
cols = [c[1] for c in cur.fetchall()]
if EXPANDED_COL not in cols:
    print(f"Adding column '{EXPANDED_COL}' to {TABLE_NAME}...")
    cur.execute(f"ALTER TABLE {TABLE_NAME} ADD COLUMN {EXPANDED_COL} TEXT;")
    conn.commit()

# Show counts
cur.execute(f"SELECT COUNT(*) FROM {TABLE_NAME};")
total_rows = cur.fetchone()[0]

cur.execute(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE {EXPANDED_COL} IS NULL;")
remaining_rows = cur.fetchone()[0]

print(f"Total rows in {TABLE_NAME}: {total_rows}")
print(f"Rows still needing expansion (expanded IS NULL): {remaining_rows}")

conn.close()


Total rows in finviz_news_highfreq_table: 10448
Rows still needing expansion (expanded IS NULL): 10448


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading TinyLlama model on GPU…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",         # run on GPU
    torch_dtype=torch.float16, # fast + memory efficient
)

MAX_NEW_TOKENS = 36  # short, efficient expansions

PROMPT_TEMPLATE = """You are a helpful financial news explainer.

Expand the following stock market news headline into 2 short, clear sentences
that explain what the news likely means for investors. Do NOT invent specific
numbers or fictional companies; just explain the general implications.

Headline: "{headline}"
Expanded explanation:"""

def expand_headline(headline: str, max_new_tokens: int = MAX_NEW_TOKENS) -> str:
    safe_headline = headline.replace('"', '\\"')
    prompt = PROMPT_TEMPLATE.format(headline=safe_headline)

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Expanded explanation:" in full_text:
        expanded = full_text.split("Expanded explanation:", 1)[1].strip()
    else:
        expanded = full_text.strip()

    return expanded


Loading TinyLlama model on GPU…


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# EXPANDING HEADLINES


import sqlite3
from tqdm.auto import tqdm

# Same config as before
DB_PATH = "/content/StockNewsProject/finviz_news.db"
TABLE_NAME = "finviz_news_highfreq_table"
ID_COL = "id"
TITLE_COL = "title"
EXPANDED_COL = "expanded"

BATCH_COMMIT = 20  # tradeoff between speed and safety

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.execute(
    f"""
    SELECT {ID_COL}, {TITLE_COL}
    FROM {TABLE_NAME}
    WHERE {EXPANDED_COL} IS NULL
    """
)
rows = cur.fetchall()
print(f"Headlines to expand in this run: {len(rows)}")

# Cache expansions by title to avoid recomputing duplicates
cache = {}

updated = 0

for row_id, title in tqdm(rows, desc="Expanding headlines"):
    title = title or ""
    title_key = title.strip()

    if not title_key:
        expanded = ""
    elif title_key in cache:
        expanded = cache[title_key]
    else:
        try:
            expanded = expand_headline(title_key)
        except Exception as e:
            print(f"\nError expanding id={row_id}, title={title!r}: {e}")
            expanded = ""
        cache[title_key] = expanded

    cur.execute(
        f"UPDATE {TABLE_NAME} SET {EXPANDED_COL} = ? WHERE {ID_COL} = ?;",
        (expanded, row_id),
    )

    updated += 1
    if updated % BATCH_COMMIT == 0:
        conn.commit()

conn.commit()
conn.close()
print(f"Done. Expanded {updated} headlines in this run.")


Headlines to expand in this run: 10448


Expanding headlines:   0%|          | 0/10448 [00:00<?, ?it/s]

Done. Expanded 10448 headlines in this run.


In [ ]:
### CHECK TO MAKE SURE


import sqlite3

DB_PATH = "/content/StockNewsProject/finviz_news.db"
TABLE_NAME = "finviz_news_highfreq_table"

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.execute(f"PRAGMA table_info({TABLE_NAME});")
cols = [row[1] for row in cur.fetchall()]

needed_cols = [
    ("sent_pos", "REAL"),
    ("sent_neu", "REAL"),
    ("sent_neg", "REAL"),
    ("sent_score", "REAL"),
]

for col_name, col_type in needed_cols:
    if col_name not in cols:
        print(f"Adding column {col_name} to {TABLE_NAME}...")
        cur.execute(f"ALTER TABLE {TABLE_NAME} ADD COLUMN {col_name} {col_type};")
        conn.commit()

# Quick status:
cur.execute(f"SELECT COUNT(*) FROM {TABLE_NAME};")
total_rows = cur.fetchone()[0]

cur.execute(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE expanded IS NOT NULL AND sent_score IS NULL;")
remaining_rows = cur.fetchone()[0]

print(f"Total rows in {TABLE_NAME}: {total_rows}")
print(f"Rows with expanded text but no sentiment yet: {remaining_rows}")

conn.close()


Adding column sent_pos to finviz_news_highfreq_table...
Adding column sent_neu to finviz_news_highfreq_table...
Adding column sent_neg to finviz_news_highfreq_table...
Adding column sent_score to finviz_news_highfreq_table...
Total rows in finviz_news_highfreq_table: 10448
Rows with expanded text but no sentiment yet: 10448


In [ ]:
### LOAD FINBERT TO GPU ###

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

FINBERT_MODEL_NAME = "ProsusAI/finbert"

print("Loading FinBERT on GPU…")
finbert_tokenizer = AutoTokenizer.from_pretrained(FINBERT_MODEL_NAME)
finbert_model = AutoModelForSequenceClassification.from_pretrained(
    FINBERT_MODEL_NAME
).to("cuda").eval()

# According to FinBERT docs: label mapping
# 0 -> neutral, 1 -> positive, 2 -> negative
FINBERT_LABELS = ["neutral", "positive", "negative"]

import torch.nn.functional as F

def finbert_score_batch(texts):
    """
    texts: list of strings
    returns: list of dicts with keys: pos, neu, neg, score
    """
    encodings = finbert_tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to("cuda")

    with torch.no_grad():
        outputs = finbert_model(**encodings)
        probs = F.softmax(outputs.logits, dim=-1).cpu().numpy()

    results = []
    for p in probs:
        neu = float(p[0])
        pos = float(p[1])
        neg = float(p[2])
        score = pos - neg  # simple polarity
        results.append(
            {"pos": pos, "neu": neu, "neg": neg, "score": score}
        )
    return results


GPU available: True
GPU name: Tesla T4
Loading FinBERT on GPU…


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
#### RUNNING FINBERT

import sqlite3
from tqdm.auto import tqdm

DB_PATH = "/content/StockNewsProject/finviz_news.db"
TABLE_NAME = "finviz_news_highfreq_table"

BATCH_SIZE = 64
BATCH_COMMIT = 64  # commit once per batch

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Fetch all rows needing sentiment
cur.execute(
    f"""
    SELECT id, expanded
    FROM {TABLE_NAME}
    WHERE expanded IS NOT NULL
      AND sent_score IS NULL
    """
)
rows = cur.fetchall()
print(f"Rows to score with FinBERT in this run: {len(rows)}")

total = len(rows)
processed = 0

for i in tqdm(range(0, total, BATCH_SIZE), desc="Scoring with FinBERT"):
    batch = rows[i:i + BATCH_SIZE]
    ids = [r[0] for r in batch]
    texts = [r[1] if r[1] is not None else "" for r in batch]

    # Replace empty strings with something minimal to avoid tokenizer issues
    texts = [t if t.strip() else "No content." for t in texts]

    scores = finbert_score_batch(texts)

    for row_id, s in zip(ids, scores):
        cur.execute(
            f"""
            UPDATE {TABLE_NAME}
            SET sent_pos = ?, sent_neu = ?, sent_neg = ?, sent_score = ?
            WHERE id = ?;
            """,
            (s["pos"], s["neu"], s["neg"], s["score"], row_id),
        )

    conn.commit()
    processed += len(batch)

conn.close()
print(f"Done. Scored {processed} rows with FinBERT.")


Rows to score with FinBERT in this run: 10448


Scoring with FinBERT:   0%|          | 0/164 [00:00<?, ?it/s]

Done. Scored 10448 rows with FinBERT.


In [ ]:
import sqlite3
import pandas as pd


In [ ]:
DB_PATH = "/content/StockNewsProject/finviz_news.db"
TABLE_NAME = "finviz_news_highfreq_table"

conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME};", conn)

conn.close()

df.head()


,id,ticker,day,time,title,source,link,expanded,sent_pos,sent_neu,sent_neg,sent_score
0,300,AAPL,2025-09-25,12:39:00,"Citadel's Ken Griffin Sounds Off on Tariffs, Q...",The Wall Street Journal,https://finance.yahoo.com/m/d8d5642f-31ce-3562...,"Griffin, the billionaire hedge fund manager an...",0.775056,0.038179,0.186765,0.588291
1,299,AAPL,2025-09-25,13:05:00,"Will Smart Glasses Replace Smartphones? Meta, ...",Investor's Business Daily,https://www.investors.com/news/technology/smar...,"Smart glasses, which are still in the early st...",0.016009,0.347475,0.636516,-0.620506
2,298,AAPL,2025-09-25,13:09:00,"Stocks to Watch Thursday: Tesla, Oracle, Intel...",The Wall Street Journal,https://finance.yahoo.com/m/ec054959-af21-36bb...,Tesla (TSLA) is expected to report earnings be...,0.052657,0.047524,0.899819,-0.847162
3,297,AAPL,2025-09-25,13:12:00,"Jim Cramer on Apple: ""I Say Over and Over, Sho...",Insider Monkey,/news/176079/jim-cramer-on-apple-i-say-over-an...,"Cramer, the host of CNBC's ""Mad Money,"" said t...",0.038445,0.046226,0.915329,-0.876885
4,296,AAPL,2025-09-25,13:23:00,Should Apple buy Intel's stock? These analysts...,MarketWatch,https://www.marketwatch.com/story/should-apple...,Analysts at Jefferies and Morgan Stanley both ...,0.023312,0.945152,0.031536,-0.008225


In [ ]:
csv_path = "/content/StockNewsProject/finviz_news_export.csv"
df.to_csv(csv_path, index=False)

print("Saved to:", csv_path)


Saved to: /content/StockNewsProject/finviz_news_export.csv


In [ ]:
import pandas as pd

# ==========================
# 1. LOAD ORIGINAL CSV
# ==========================
df = pd.read_csv(csv_path)

# ==========================
# 2. CLEAN DATE + FILTER INVALID ROWS
# ==========================
df["day"] = pd.to_datetime(df["day"], errors="coerce")

# Drop rows with no ticker or no date
df = df.dropna(subset=["day", "ticker"])

# ==========================
# 3. DAILY AGGREGATION (ALL SENTIMENT COLUMNS)
# ==========================

daily_sentiment = (
    df.groupby(["ticker", "day"], as_index=False)[
        ["sent_pos", "sent_neu", "sent_neg", "sent_score"]
    ]
    .mean()
)

# Sort for cleanliness
daily_sentiment = daily_sentiment.sort_values(["ticker", "day"])

print("Preview of aggregated daily sentiment:")
print(daily_sentiment.head())

# ==========================
# 4. SAVE TO CSV
# ==========================
output_csv = "daily_sentiment_full.csv"
daily_sentiment.to_csv(output_csv, index=False)

print(f"\nSaved full daily aggregated sentiment to: {output_csv}")


Preview of aggregated daily sentiment:
  ticker        day  sent_pos  sent_neu  sent_neg  sent_score
0   AAPL 2025-09-25  0.231947  0.432750  0.335303   -0.103355
1   AAPL 2025-09-26  0.092287  0.645962  0.261751   -0.169463
2   AAPL 2025-09-27  0.163220  0.586011  0.250769   -0.087549
3   AAPL 2025-09-28  0.085521  0.437715  0.476764   -0.391242
4   AAPL 2025-09-29  0.431803  0.310577  0.257620    0.174183

Saved full daily aggregated sentiment to: daily_sentiment_full.csv


In [ ]:
import yfinance as yf
import pandas as pd

# ==========================
# CONFIG
# ==========================

TICKERS = [
    'AAPL', 'MSFT', 'NVDA', 'GOOGL', 'META',
    'AMZN', 'TSLA', 'JPM'
]

START_DATE = "2025-09-25"
END_DATE   = "2025-12-02"  # yfinance end is EXCLUSIVE, so 12/02 includes 12/01
OUTPUT_CSV = "stock_daily_2025-09-25_to_2025-12-01.csv"

# ==========================
# DOWNLOAD DAILY DATA
# ==========================

print("Downloading daily price data...")

data = yf.download(
    tickers=TICKERS,
    start=START_DATE,
    end=END_DATE,
    interval="1d",
    group_by="column",     # important: makes first level the field (Open, High, etc.)
    auto_adjust=False,
    back_adjust=False,
    actions=True,
    threads=True,
    progress=True
)

# ==========================
# FLATTEN MULTIINDEX TO LONG FORMAT
# ==========================

if isinstance(data.columns, pd.MultiIndex):
    # data.columns: level 0 = field (Open, High, ...), level 1 = ticker
    # We want: Date, Ticker, Open, High, Low, Close, Adj_Close, Volume, Dividends, Stock_Splits

    # Swap so level 0 = ticker, level 1 = field, then stack tickers
    data = data.swaplevel(0, 1, axis=1)  # now columns.levels[0] = tickers, [1] = fields
    data = data.sort_index(axis=1)

    # Stack by ticker to get a long DataFrame
    df = data.stack(level=0).reset_index()
    df = df.rename(columns={"level_1": "Ticker"})
else:
    # Single-ticker case (unlikely here, but for completeness)
    df = data.reset_index()
    df["Ticker"] = TICKERS[0]

# Rename columns to standard names
df = df.rename(columns={
    "Adj Close": "Adj_Close",
    "Stock Splits": "Stock_Splits"
})

# ==========================
# SELECT & CLEAN COLUMNS
# ==========================

keep = [
    "Date", "Ticker",
    "Open", "High", "Low", "Close", "Adj_Close",
    "Volume", "Dividends", "Stock_Splits"
]

# Ensure all columns exist (in case some are missing for certain tickers)
for col in keep:
    if col not in df.columns:
        df[col] = None

df = df[keep].copy()

# Sort nicely
df = df.sort_values(["Ticker", "Date"], kind="mergesort").reset_index(drop=True)

print("Sample of final daily data:")
print(df.head())
print(df.tail())

# ==========================
# SAVE TO CSV
# ==========================

df.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved daily CSV to: {OUTPUT_CSV}")


[*********************100%***********************]  8 of 8 completed

Sample of final daily data:
Price       Date Ticker        Open        High         Low       Close  \
0     2025-09-25   AAPL  253.210007  257.170013  251.710007  256.869995   
1     2025-09-26   AAPL  254.100006  257.600006  253.779999  255.460007   
2     2025-09-29   AAPL  254.559998  255.000000  253.009995  254.429993   
3     2025-09-30   AAPL  254.860001  255.919998  253.110001  254.630005   
4     2025-10-01   AAPL  255.039993  258.790009  254.929993  255.449997   

Price   Adj_Close    Volume  Dividends  Stock_Splits  
0      256.621216  55202100        0.0           0.0  
1      255.212601  46076300        0.0           0.0  
2      254.183594  40127700        0.0           0.0  
3      254.383408  37704300        0.0           0.0  
4      255.202606  48713900        0.0           0.0  
Price       Date Ticker        Open        High         Low       Close  \
371   2025-11-24   TSLA  402.170013  421.720001  401.089996  417.779999   
372   2025-11-25   TSLA  414.420013  420.


/tmp/ipython-input-2631827235.py:49: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = data.stack(level=0).reset_index()


In [ ]:
import pandas as pd

# ==========================
# 1. LOAD BOTH CSV FILES
# ==========================

stock_df = pd.read_csv("/content/StockNewsProject/stock_daily_2025-09-25_to_2025-12-01.csv")
sent_df  = pd.read_csv("/content/StockNewsProject/daily_sentiment_full.csv")

# ==========================
# 2. CLEAN + ALIGN COLUMN NAMES
# ==========================

# Ensure Date is datetime
stock_df["Date"] = pd.to_datetime(stock_df["Date"])
sent_df["day"]   = pd.to_datetime(sent_df["day"])

# Rename columns so they match
sent_df = sent_df.rename(columns={"day": "Date", "ticker": "Ticker"})

# ==========================
# 3. LEFT JOIN (stock data = left)
# ==========================

merged = pd.merge(
    stock_df,
    sent_df,
    on=["Ticker", "Date"],
    how="left"          # LEFT JOIN → keep all rows from stock_df
)

print("Merged preview:")
print(merged.head())

# ==========================
# 4. SAVE OUTPUT CSV
# ==========================

output_csv = "merged_stock_sentiment.csv"
merged.to_csv(output_csv, index=False)

print(f"\nSaved merged dataset to: {output_csv}")


Merged preview:
        Date Ticker        Open        High         Low       Close  \
0 2025-09-25   AAPL  253.210007  257.170013  251.710007  256.869995   
1 2025-09-26   AAPL  254.100006  257.600006  253.779999  255.460007   
2 2025-09-29   AAPL  254.559998  255.000000  253.009995  254.429993   
3 2025-09-30   AAPL  254.860001  255.919998  253.110001  254.630005   
4 2025-10-01   AAPL  255.039993  258.790009  254.929993  255.449997   

    Adj_Close    Volume  Dividends  Stock_Splits  sent_pos  sent_neu  \
0  256.621216  55202100        0.0           0.0  0.231947  0.432750   
1  255.212601  46076300        0.0           0.0  0.092287  0.645962   
2  254.183594  40127700        0.0           0.0  0.431803  0.310577   
3  254.383408  37704300        0.0           0.0  0.375938  0.404933   
4  255.202606  48713900        0.0           0.0  0.272818  0.385747   

   sent_neg  sent_score  
0  0.335303   -0.103355  
1  0.261751   -0.169463  
2  0.257620    0.174183  
3  0.219129    0.156

In [ ]:
import pandas as pd

# ==========================
# 1. LOAD MERGED DATA
# ==========================
merged = pd.read_csv("/content/StockNewsProject/merged_stock_sentiment.csv")

# Ensure Date is datetime
merged["Date"] = pd.to_datetime(merged["Date"], errors="coerce")

# Sort by ticker and date
merged = merged.sort_values(["Ticker", "Date"])

# ==========================
# 2. FILL MISSING SENTIMENT PER TICKER
# ==========================

sent_cols = ["sent_pos", "sent_neu", "sent_neg", "sent_score"]

# Forward-fill then backward-fill within each ticker
merged[sent_cols] = (
    merged
    .groupby("Ticker")[sent_cols]
    .ffill()
    .bfill()
)

# OPTIONAL: if any ticker has *no* sentiment at all (rare),
# you can default remaining NaNs to 0 (neutral sentiment):
# merged[sent_cols] = merged[sent_cols].fillna(0.0)

print("Preview after filling sentiment:")
print(merged.head())

# ==========================
# 3. SAVE TO NEW CSV
# ==========================
output_csv = "merged_stock_sentiment_filled.csv"
merged.to_csv(output_csv, index=False)

print(f"\nSaved filled dataset to: {output_csv}")


Preview after filling sentiment:
        Date Ticker        Open        High         Low       Close  \
0 2025-09-25   AAPL  253.210007  257.170013  251.710007  256.869995   
1 2025-09-26   AAPL  254.100006  257.600006  253.779999  255.460007   
2 2025-09-29   AAPL  254.559998  255.000000  253.009995  254.429993   
3 2025-09-30   AAPL  254.860001  255.919998  253.110001  254.630005   
4 2025-10-01   AAPL  255.039993  258.790009  254.929993  255.449997   

    Adj_Close    Volume  Dividends  Stock_Splits  sent_pos  sent_neu  \
0  256.621216  55202100        0.0           0.0  0.231947  0.432750   
1  255.212601  46076300        0.0           0.0  0.092287  0.645962   
2  254.183594  40127700        0.0           0.0  0.431803  0.310577   
3  254.383408  37704300        0.0           0.0  0.375938  0.404933   
4  255.202606  48713900        0.0           0.0  0.272818  0.385747   

   sent_neg  sent_score  
0  0.335303   -0.103355  
1  0.261751   -0.169463  
2  0.257620    0.174183  
3  